In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="asdf1122"
)
cursor = conn.cursor()

# DB 생성
cursor.execute("CREATE DATABASE IF NOT EXISTS wconcept CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
conn.database = "wconcept"

cursor.execute("""
CREATE TABLE IF NOT EXISTS categories (
    category_code VARCHAR(50) PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS performances (
    performance_id INT AUTO_INCREMENT PRIMARY KEY,
    category_code VARCHAR(50),
    title VARCHAR(200),
    rate VARCHAR(50),
    review_num VARCHAR(50),
    UNIQUE KEY unique_performance (category_code, title, rate, review_num),
    FOREIGN KEY (category_code) REFERENCES categories(category_code) ON DELETE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS reviews (
    review_id INT AUTO_INCREMENT PRIMARY KEY,
    performance_id INT,
    review_text TEXT,
    FOREIGN KEY (performance_id) REFERENCES performances(performance_id) ON DELETE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
""")

conn.commit()

category_map = {
    "001001001": "자켓",
    "001001002": "코트",
    "001001008": "트렌치코트",
    "001001010": "핸드메이드코트",
    "001001003": "점퍼",
    "001001011": "후드집업",
    "001001007": "패딩"
}

# 데이터
for code, name in category_map.items():
    cursor.execute("""
        INSERT IGNORE INTO categories (category_code, category_name)
        VALUES (%s, %s)
    """, (code, name))
conn.commit()

# DB 저장
def save_performance(category_code, title, rate, review_num, reviews):
    sql_perf = """
    INSERT IGNORE INTO performances (category_code, title, rate, review_num)
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(sql_perf, (category_code, title, rate, review_num))
    conn.commit()

    cursor.execute("""
        SELECT performance_id FROM performances
        WHERE category_code=%s AND title=%s AND rate=%s AND review_num=%s
    """, (category_code, title, rate, review_num))
    performance_id = cursor.fetchone()[0]

    for review in reviews:
        sql_review = """
        INSERT INTO reviews (performance_id, review_text)
        VALUES (%s, %s)
        """
        cursor.execute(sql_review, (performance_id, review))
    conn.commit()

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")
driver = webdriver.Chrome(service=service, options=options)

for category_code, category_name in category_map.items():
    url = f"https://display.wconcept.co.kr/category/women/{category_code}"
    driver.get(url)
    time.sleep(2)
    print(f"\n\n========== {category_name} ({category_code}) ==========")

    idx = 0
    while idx < 30:
        try:
            buttons = driver.find_elements(By.CSS_SELECTOR, "button.area-img")
            if idx >= len(buttons):
                break

            driver.execute_script("arguments[0].click();", buttons[idx])
            time.sleep(2)
            print(f"\n----- {category_name} {idx+1}번째 의류 -----")

            # 상품명
            try:
                title = driver.find_element(By.CSS_SELECTOR, "h2.brand").text.strip()
            except NoSuchElementException:
                title = "상품명 없음"

            # 평점
            try:
                rate = driver.find_element(By.CSS_SELECTOR, "p.grade_num").text.strip()
            except NoSuchElementException:
                rate = "0"

            # 리뷰수
            try:
                review_num = driver.find_element(By.CSS_SELECTOR, "#reviewCnt1").text.strip()
            except NoSuchElementException:
                review_num = "0"

            # 리뷰 최대 3개
            reviews = []
            try:
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(0.5)
                review_button = driver.find_element(By.CSS_SELECTOR, "#detail > li:nth-child(2)")
                review_button.click()
                time.sleep(1.5)

                review_elements = driver.find_elements(By.CSS_SELECTOR, "p.pdt_review_text")
                reviews = [r.text.strip() for r in review_elements[:3]]
                if not reviews:
                    reviews = ["댓글 정보 없음"]
            except NoSuchElementException:
                reviews = ["댓글 정보 없음"]

            print("상품명:", title)
            print("평점:", rate)
            print("리뷰수:", review_num)
            for i, review in enumerate(reviews, start=1):
                print(f"리뷰{i}:", review)

            # DB 저장
            save_performance(category_code, title, rate, review_num, reviews)

            driver.back()
            time.sleep(0.5)
            driver.back()
            time.sleep(1)
            driver.execute_script("window.scrollBy(0, 100);")
            time.sleep(1)

            idx += 1
        except StaleElementReferenceException:
            print(f"{idx+1}번째 요소 stale 참조 발생, 재시도")
            time.sleep(1)
            continue

driver.quit()
cursor.close()
conn.close()



========== 자켓 (001001001) ==========

----- 자켓 1번째 의류 -----
상품명: Dunst for WOMEN
평점: 5
리뷰수: 11
리뷰1: 피부가 까매서 옷 색상에 되게 민감한데 예전부터 찾아헤매던 카키색이 맞아요! 고급스러워 보이는 컬러고 가을 겨울에 찰떡이에요 내피 배색도 잘 되어있어서 어디가서 옷벗어놓아도 좋아요~ 다만 큰박스 입은거처럼 왜이렇게 뻣뻣하고 부스럭 거릴까요.. 글 중에 왁스처리? 해놨단 말이 있었는데 그래서 그런가봐요 촤르르 부들부들 좋아하는 스퇄이라 조금 아쉬워요 사이즈는 s가 66이란말에 xs했는데 넉넉하게 잘 맞아요 키가 작은데 팔길이도 조금 길다 싶을정도로 맞네요~ 캐주얼하게 점퍼형태 옷이랑 고민하다가 세일해서 던스트로 샀는데 아주 만족합니다
리뷰2: 사이즈 고민하다 S어요 오버핏 조아해서 M으로 할까하다가 던스트옷들이 좀 크게 나와서 걍S로했는데 M했으면 너무 클뻔했어요 옷도 빳빳하니 이쁘네요
리뷰3: 네이비 샀는데
네이비보다 검정에 가까워요! 예쁘고 무난하게 입기 좋습니다~!
빨리 가을 오면 좋겠어요

----- 자켓 2번째 의류 -----
상품명: IvanaHelsinki단독
평점: 4.9
리뷰수: 610
리뷰1: 디자인이랑 컬러는 사진이랑 똑같아요 맘에듬!
사이즈는 평소에 s사이즈 자켓입는데 작은듯하면서 따악 맞는느낌이라 요즘 살이좀 쪘나싶은 ㅎㅎ 옷이이쁘니까 그냥 조금만 빼서 입으면 될듯용 하나큰거샀음 좀 애매했을거같아요!
리뷰2: 상견례용으로 구입했는데 옷은 예뻐요
근데 다들 어깨뽕 어떻게 하셨나요? 잘 맞으시나요..?
리뷰3: 가슴이 좀 큰 편이라 플러스 사이즈 구매했는데 불편하지 않고 좋아요!! 스커트와 함께 입으니 정말 예뻐요~

----- 자켓 3번째 의류 -----
상품명: MangoManyPlease
평점: 0
리뷰수: 0
리뷰1: 댓글 정보 없음

----- 자켓 4번째 의류 -----
상품명: LOOKAST
평점: 0
리뷰수: 0
리뷰1: 댓글 정보 없음

-----

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time, re, pandas as pd

# 크롤러 세팅
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://display.wconcept.co.kr/rn/best?displayCategoryType=10101&gnbType=Y"
driver.get(url)
time.sleep(3)

# 스크롤 내려서 상품 로드
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 상품 리스트 가져오기
products = driver.find_elements(By.CSS_SELECTOR, "div.product-item.item.type-all")[:5]  # 테스트로 5개만

all_data = []

# 상품별 반복
for product in products:
    try:
        name = product.find_element(By.CSS_SELECTOR, "span.prdc-title span.text.detail").text.strip()
        
        # 상품 ID 추출
        img = product.find_element(By.CSS_SELECTOR, "button.area-img img")
        src = img.get_attribute("src")
        m = re.search(r"/(\d+)(?:[_\.][^/?]+)?(?:\?.*)?$", src)
        pid = m.group(1) if m else None
        prod_url = f"https://www.wconcept.co.kr/Product/{pid}" if pid else ""
        
        # 상세페이지 접속
        driver.execute_script("window.open(arguments[0]);", prod_url)  # 새 탭 열기
        driver.switch_to.window(driver.window_handles[-1])  # 새 탭으로 전환
        time.sleep(2)

        # 리뷰 개수
        try:
            review_info = driver.find_element(By.CSS_SELECTOR, "#reviewCnt2").text
            review_count = int(re.sub(r"[^\d]", "", review_info)) if review_info else 0
        except:
            review_count = 0

        # 평점
        try:
            rating_text = driver.find_element(By.CSS_SELECTOR, "span.area-info em.score").text
            avg_rating = float(re.findall(r"[0-9.]+", rating_text)[0])
        except:
            avg_rating = 0.0

        # 리뷰 탭 클릭
        try:
            review_tab = driver.find_element(By.XPATH, "//li[a[contains(text(),'REVIEW')]]")
            review_tab.click()
            time.sleep(2)

            review_elements = driver.find_elements(By.CSS_SELECTOR, "p.pdt_review_text")
            reviews = [r.text.strip() for r in review_elements[:3]]  # 앞 3개만
            if not reviews:
                reviews = ["NO reviews"]
        except:
            reviews = ["NO reviews"]

        all_data.append({
            "상품명": name,
            "URL": prod_url,
            "리뷰수": review_count,
            "평균평점": avg_rating,
            "리뷰내용": reviews
        })

        # 탭 닫고 다시 리스트로 복귀
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)

    except Exception as e:
        print("에러:", e)
        continue

driver.quit()

# DataFrame 변환
df = pd.DataFrame(all_data)
print(df)